# Setup

In [1]:
import create_models
from pathlib import Path
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# Models

## Initialization

In [2]:
image_size = 64

mean_variance = create_models.mean_variance()
diffusion = create_models.spaced_diffusion()
regressor = create_models.regressor()
classifier = create_models.classifier()

## Loading Checkpoints

In [ ]:
checkpoints_path = Path(r".\checkpoints")
cpu = torch.cpu.current_device()
gpu = torch.cuda.current_device()
def get_state_dict(path: Path):
    return torch.load(path, map_location=cpu, weights_only=True)

mean_variance_path = checkpoints_path / "diff_checkpoint" / "model_180000.pt"
mean_variance.load_state_dict(get_state_dict(mean_variance_path))
mean_variance.to(gpu)
mean_variance.convert_to_fp16()
mean_variance.eval()

regressor_path = checkpoints_path / "reg_checkpoint" / "model_350000.pt"
regressor.load_state_dict(get_state_dict(regressor_path))
regressor.to(gpu)
regressor.eval()

classifier_path = checkpoints_path / "class_checkpoint" / "model_299999.pt"
classifier.load_state_dict(get_state_dict(classifier_path))
classifier.to(gpu)
classifier.eval()

# Sampling

In [8]:
batch_size = 1
channel_count = 1
shape = (batch_size, channel_count, image_size, image_size)

def cond_fn_1(x: torch.Tensor, time_steps: torch.Tensor):
    with torch.enable_grad():
        x_in = x.detach().requires_grad_()
        logits = regressor(x_in, time_steps)
        grad = torch.autograd.grad(logits.sum(), x_in)[0]
        return (-1) * grad[:,0,:,:].reshape(shape) * 4.0

def cond_fn_2(x: torch.Tensor, time_steps: torch.Tensor):
    with torch.enable_grad():
        x_in = x.detach().requires_grad_()
        logits: torch.Tensor = classifier(x_in, time_steps)
        log_probs = F.log_softmax(logits, dim=-1)
        selected = log_probs[range(len(logits)), 1]
        grad = torch.autograd.grad(selected.sum(), x_in)[0]
        return grad[:,0,:,:].reshape(shape) * 3.0


def get_boundary_condition(condition_name: str):
    folder = Path(r".\data\dataset_1_diff\test_data_level_1")
    path = folder / f"cons_{condition_name}_array_200.npy"
    ndarray = np.transpose(np.load(path), [2, 0, 1]).astype(np.float32)
    tensor = torch.unsqueeze(torch.as_tensor(ndarray), 0) # Add batch size dimension
    return tensor.to(gpu)

sample = diffusion.p_sample_loop(
    model=mean_variance,
    shape=shape,
    cons=get_boundary_condition("pf"),
    loads=get_boundary_condition("load"),
    BCs=get_boundary_condition("bc"),
    noise=None,
    clip_denoised=True,
    denoised_fn=None,
    cond_fn_1=cond_fn_1,
    cond_fn_2=cond_fn_2,
    model_kwargs={},
    device=gpu,
    progress=False,
)

In [ ]:
plt.imshow(sample[0,0,:,:].cpu().detach().numpy(), cmap="gray")